<a href="https://colab.research.google.com/github/barkhaneum/Transformer_study/blob/main/chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount("/content/drive")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset
cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail",
    "3.0.0",
    cache_dir="/content/drive/MyDrive/Transformer_study_datasets"
)

Old caching folder /content/drive/MyDrive/Transformer_study_datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f for dataset cnn_dailymail exists but no data were found. Removing it. 


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
sample = dataset['train'][1] # highlights, article, id
print(f"""기사 (500개 문자 발췌, 총 길이 : {len(sample['article'])} ) :""")
print(sample['article'][:500]) # 기사 내용 발췌
print(f'\n요약 ( 길이 : {len(sample["highlights"])}) :')
print(sample['highlights'])


In [ ]:
sample_text = dataset['train'][1]['article'][:2000] # 기사 내용 발췌
summaries = {}

In [28]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")
nltk.download("punkt")

string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization.']

In [ ]:
def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3]) # 텍스트중 맨처음 문장 세개를 선택

summaries["baseline"] = three_sentence_summary(sample_text)

In [ ]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline("text_generation", model = "gpt2-xl")
gpt2_query = sample_text + "\nTl;DR:\n"

pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces = True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))

In [ ]:
pipe = pipeline("summarization", model = "t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_test"]))

In [ ]:
pipe = pipeline("summarization", model = "facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))